# Import the necessary libraries.

In [ ]:
# Built-in packages
import json
import warnings
import re
warnings.filterwarnings("ignore")

# Third party packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import model_selection

from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor



sns.set(context= "notebook", color_codes=True)
plt.style.use('bmh')

%matplotlib inline
pd.set_option('display.max_columns', None)

## Read the data as a data frame

In [ ]:
df = pd.read_csv("/kaggle/input/regression-with-neural-networking/concrete_data.csv")
df.columns = ['cement', 'slag', 'flyash', 'water', 'superplasticizer', 'coarse_agg', 'fine_agg', 'age', 'strength']
df.head()

In [ ]:
print(f"The shape of the DataFrame is: {df.shape}, which means there are {df.shape[0]} rows and {df.shape[1]} columns.")

**cement**$~~~~~~~~~~~~~~~~~~~~~~~$- measured in kg in a m3 mixture
____
**slag**$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$- measured in kg in a m3 mixture
____
**flyash**$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$- measured in kg in a m3 mixture
____
**water**$~~~~~~~~~~~~~~~~~~~~~~~~~~$- measured in kg in a m3 mixture
____
**superplasticizer**$~~~~~~~~~~~~~~~$- measured in kg in a m3 mixture
____
**coarse_agg**$~~~~~~~~~~~~~~~~~~$- measured in kg in a m3 mixture
____
**fine_agg**$~~~~~~~~~~~~~~~~~~~~~~~$- measured in kg in a m3 mixture
____
**age**$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$- day (1~365)
____
**strength**$~~~~~~~~~~~~~~~~~~~~~$- measured in kg in a m3 mixture
____

In [ ]:
df.info()

There are 8 columns with float datatype and one column with int. There seem to be no null values in any of the columns

In [ ]:
df_summary = df.describe()
df_summary

The columns **slag**, **ash** and **superplasticizer** have zero values which could be thought of as missing values and fill them using regression.

## Check for outliers

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (20,20));
col_cnt = 0
col_names = df.columns.tolist()
for r in range(3):
    for c in range(3):
        try:
            sns.boxplot(df[col_names[col_cnt]], ax=axes[r][c], orient="h");
            col_cnt += 1
        except:
            pass

There are a few outliers in **slag**, **water**, **superplasticizer**, **fine_agg** and **age**. 

Removing the rows with outliers removes around 10% of the data so we will substitute them with the mean values.

In [ ]:
outlier_cols = ["slag", "water", "superplasticizer", "fine_agg", "age"]
def cap_outliers(df, col):
    q1 = df_summary[col].loc["25%"]
    q3 = df_summary[col].loc["75%"]
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    df.loc[df[col] < lower_bound, col] = df_summary[col].loc["mean"]
    df.loc[df[col] > upper_bound, col] = df_summary[col].loc["mean"]
    return df
    
    
for col in outlier_cols:
    df = cap_outliers(df, col)

## Multivariate analysis

In [ ]:
sns.pairplot(df, diag_kind="kde");

In [ ]:
fig, axes = plt.subplots(1, 3, figsize = (20,10));

sns.scatterplot(y="slag", x="cement", data=df, ax=axes[0], alpha=0.5);
sns.scatterplot(y="flyash", x="cement", data=df, ax=axes[1], alpha=0.5);
sns.scatterplot(y="superplasticizer", x="cement", data=df, ax=axes[2], alpha=0.5);

In the charts above, we see some sort of pattern in the data and a lot of columns with zero values. We shall impute them using Linear Regression by considering other features to predict them.

**Observations:**

* As the amount of cement increases, the amount of slag does too. A pattern is visible in the first chart.
* If we look at the cement vs flyash closely, we can see pattern with a negative slope, the amount of cement is inversely proportional to the amount of flyash.
* The cement vs superplasticizer trend is similar to that of cement vs slag.

### Perform necessary imputation

In [ ]:
def impute_values(alg, df, a, cols):
    df[a] = df[a].replace({0: np.nan})
    df_notnull = df[df[a].notna()]
    
    X = df_notnull[cols]
    y = df_notnull[a]
    
    df_null = df[df[a].isna()]
    test_X = df_null[cols]

    regr = alg
    regr.fit(X, y)

    pred = regr.predict(test_X)
    df_null = df[df[a].isna()]
    df_null[a] = pred
    
    xx = df[df[a].notna()]
    xx["null"] = 0
    df_null["null"] = 1
    df = pd.concat([xx, df_null], axis=0)
    cols.append(a)
    return df

In [ ]:
for col, params in {"flyash": ["cement", "water", "fine_agg"], "slag": ["cement", "coarse_agg", "fine_agg"], "superplasticizer" : ["cement", "water", "coarse_agg"]}.items():
    alg = LinearRegression()
    df = impute_values(alg, df, col, params)
    
fig, axes = plt.subplots(1, 3, figsize = (20,10));

sns.scatterplot(y="slag", x="cement", data=df, hue="null", ax=axes[0], alpha=0.5, palette=["steelblue", "green"]);
sns.scatterplot(y="flyash", x="cement", data=df, hue="null", ax=axes[1], alpha=0.5, palette=["steelblue", "green"]);
sns.scatterplot(y="superplasticizer", x="cement", data=df, hue="null", ax=axes[2], alpha=0.5, palette=["steelblue", "green"]);
plt.show()

The above plot shows the zero values imputed in green and the rest in blue. Looks like our imputed values fit the patterns.

In [ ]:
df.drop(columns=["null"], inplace=True)

In [ ]:
sns.jointplot(x="flyash", y="slag", data=df[["flyash", "slag"]], kind="reg");

For some of the data points in the above plot, there seems to be a direct correlation between the amount of slag and flyash with a slope of almost equal to 0.8 which is quite high. Amount of slag increases with increase in flyash.

# Feature engineering

### Creating composite features

We can create 2 composite features using **cement** - **water** and **coarse_agg** - **fine_agg**

In [ ]:
df_copy = df.copy()
df_copy["cement_water_ratio"] = df_copy["cement"]/df_copy["water"]
df_copy["average_agg"] = (df_copy["coarse_agg"] + df_copy["fine_agg"])/2
df_copy.drop(columns=["cement", "water", "coarse_agg", "fine_agg"], inplace=True)

In [ ]:
target_col = ["strength"]
col_names = df_copy.columns.to_list()
col_names.remove(target_col[0])

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(df_copy[col_names + target_col].corr(), fmt=".2g", annot=True);

* It looks like our composite feature **cement_water_ratio** and **flyash** are negatively correlated. 
* The target column **strength** has good amount of correlation with **age** and **cement_water_ratio**.

### Explore for Gaussians

In [ ]:
g = sns.PairGrid(df_copy, diag_sharey=False)
g.map_upper(sns.scatterplot, s=15)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=2)

* Looking at the kdeplots of strength vs other independent columns, we can see that the data is infact a mixture of Gaussians.

* Strength vs flyash shows us two distinct Gaussians and 2 clusters but looking at the position of the two clusters, it might not be very helpful in determining/predicting the strength and the same goes for Strength vs superplasticizer.

In [ ]:
sns.jointplot(x="strength", y="age", data=df_copy[["age", "strength"]], kind="kde");

* Strength vs age is very interesting to look at. We can clearly see four clusters, 2 big ones and 2 tiny. We can also see that as the age increases, the strength increases too so this might be a helpful feature in predicting the Strength. There is one cluster that is far away from the rest of the clusters. It could mean that the concrete that has an age range of 80-100 has average strength.

* We will check if it really works by creating a KNeighbors Regressor model.

## Create the models

In [ ]:
# Prepare data for training

X = df_copy[col_names]    # Contains the independent columns 
y = df_copy[target_col]     # Our target column

seed = 16
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = seed)
train_X_poly, train_y_poly, test_X_poly, test_y_poly = (None,) * 4
train_y = train_y[target_col[0]]
test_y = test_y[target_col[0]]

In [ ]:
evaluation = {"Metrics" : ['Root Mean Squared Error (RMSE)', 'Mean Absolute Error (MAE)', 'Mean Squared Error (MSE)', 'R2 score', 'CV-mean']}
def evaluate_model(name, train_X, train_y, test_X, test_y, types, alg, plot=True):
    global evaluation, pred
    
    alg.fit(train_X, train_y)
    print(f"Score: {alg.score(test_X, test_y)}")
    if plot:
        fig, axes = plt.subplots(2, 1, figsize=(20, 10))
        try:
            if types == "Coefs":
                print(f"Intercept: {alg.intercept_}")
                try:
                    coefs = pd.DataFrame({"coefs" : alg.coef_, "col" : col_names})
                except:
                    coefs = pd.DataFrame({"coefs" : alg.coef_[0], "col" : col_names})
                sns.barplot(x="col", y="coefs", data=coefs, ax=axes[1]);
            else:
                features = pd.DataFrame({"features" : alg.feature_importances_, "col" : col_names})
                sns.barplot(x="col", y="features", data=features, ax=axes[1]);
        except:
            pass
    else:
        plt.figure(figsize=(20,5));
        axes = [None]

    pred = alg.predict(test_X)
    rmsecm = np.sqrt(metrics.mean_squared_error(test_y,pred))
    mae = mean_absolute_error(test_y, pred)
    mse = mean_squared_error(test_y, pred)
    r2 = r2_score(test_y, pred)

    p = pd.DataFrame(pred, columns=[0])
    p["Type"] = "Predictions"
    p["n"] = list(range(p.shape[0]))
    t = test_y.copy()
    t = t.reset_index().set_index("index")
    t.columns = [0]
    t["Type"] = "Actual"
    t = t[t[0] != "Actual"]
    t["n"] = list(range(p.shape[0]))
    x = pd.concat([p,t], axis=0).reset_index()
    sns.lineplot(x="n", y=0, hue="Type", data=x, markers=["o", "o"], style="Type", ax=axes[0]);
    
    cv = cross_val_score(alg, X, y, cv=10)
    cv_mean = cv.ravel().mean()
    
    evaluation[name] = [rmsecm, mae, mse, r2, cv_mean]
    df_ev = pd.DataFrame(evaluation)
    plt.show()
    return df_ev, cv

### Multiple linear regression

In [ ]:
lr = linear_model.LinearRegression()
evaluation, cv_scores = evaluate_model("Multiple Regression", train_X, train_y, test_X, test_y, "Coefs", lr)
evaluation.set_index("Metrics")

The column **cement_water_ratio** has the highest coefficient value than the rest of the features which means it has the highest weight in the prediction of the **strength** column.

___________

In [ ]:
print(cv_scores)
sns.distplot(cv_scores);
print(f"Average score: {round(cv_scores.mean(),2)*100}%")

### Ridge regressor

In [ ]:
rg = linear_model.Ridge()
evaluation, cv_scores = evaluate_model("Ridge", train_X, train_y, test_X, test_y, "Coefs", rg)
evaluation.set_index("Metrics")

The column **cement_water_ratio** has the highest coefficient value than the rest of the features which means it has the highest weight in the prediction of the **strength** column.

___________

## Decide on complexity of the model

### Polynomial regression

In [ ]:
# Fitting Polynomial Regression to the dataset

poly_reg = PolynomialFeatures(degree=3)
X_poly = poly_reg.fit_transform(X)

train_X_poly, test_X_poly, train_y_poly, test_y_poly = train_test_split(X_poly, y, test_size = 0.3, random_state = seed)
train_y_poly = train_y_poly[target_col[0]]
test_y_poly = test_y_poly[target_col[0]]

plr = LinearRegression()
evaluation, cv_scores = evaluate_model("Polynomial Regression", train_X_poly, train_y_poly, test_X_poly, test_y_poly,  "Coefs", plr, plot=False)
evaluation.set_index("Metrics")

In [ ]:
print(cv_scores)
sns.distplot(cv_scores);
print(f"Average score: {round(cv_scores.mean(),2)*100}%")

##### Looking at the cross validation scores of both Multiple linear regressor and Polynomial regressor with parameter of higher degrees, they are exactly the same so we don't really need a complex model. Also, looking at the coefficients, it looks like the strength can be calculated with a simple linear equation using these coefficients (meaning it is made up simple proportions of the other features). 
##### On a separate not, it doesn't give us any extra accuracy, instead increases complexity and time required to give the same results.

___________

## Support Vector Regressor

In [ ]:
svr = SVR(C=10, kernel="linear")
evaluation, cv_scores = evaluate_model("Support Vector Regression", train_X, train_y, test_X, test_y, "Coefs", svr)
evaluation.set_index("Metrics")

In [ ]:
knn = KNeighborsRegressor(n_neighbors=4, metric="manhattan", weights="distance")
evaluation, cv_scores = evaluate_model("KNN", train_X, train_y, test_X, test_y, "Coefs", knn, plot=False)
evaluation.set_index("Metrics")

In [ ]:
dtr = DecisionTreeRegressor()
evaluation, cv_scores = evaluate_model("Decision Tree Regression", train_X, train_y, test_X, test_y, "Features", dtr)
evaluation.set_index("Metrics")

It is clearly visible that the feature that have the most importance are the **cement_water_ratio** and **age** in predicting the **strength**.

___________

In [ ]:
rfr = RandomForestRegressor()
evaluation, cv_scores = evaluate_model("Random Forest Regression", train_X, train_y, test_X, test_y, "Features", rfr)
evaluation.set_index("Metrics")

It is clearly visible that the feature that have the most importance are the **cement_water_ratio** and **age** in predicting the **strength**.

___________

In [ ]:
xgc = XGBRegressor()
evaluation, cv_scores = evaluate_model("XGBoost", train_X, train_y, test_X, test_y, "Features", xgc)
evaluation.set_index("Metrics")

It is clearly visible that the feature that have the most importance are the **cement_water_ratio** and **age** in predicting the **strength**. Though **age** has almost half the importance as **cement_water_ratio**.

___________

#### Let's make use of GridSearchCV to find the right set of parameters for our model

In [ ]:
model = XGBRegressor(n_jobs=4)

parameters = {
    'n_estimators': [50, 100, 500],
    'max_depth': [2, 4, 6, 8, 10],
    'gamma': [0.001, 0.01],
    'learning_rate': [0.01, 0.1],
    'booster': ['gbtree']
}

grid_obj = GridSearchCV(
    estimator=xgc,
    param_grid=parameters
)

grid_obj.fit(X, y)

best_model = grid_obj.best_estimator_
print(best_model)

In [ ]:
xgc = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.001, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, monotone_constraints='()',
             n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
evaluation, cv_scores =  evaluate_model("XGBoost Tuned", train_X, train_y, test_X, test_y, "Features", xgc)
evaluation.set_index("Metrics")

Please consider upvoting if you found this useful and informative. Thank you.